In [2]:
import pandas as pd
import re

#df : 스크래핑한 호주 문서
df = pd.read_csv("호주_세관_최근10.csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

print(len(df_add))

#df_keyword : 선정한 호주 키워드 101개
df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)
print(len(df_keyword))

10
102


In [3]:
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# SentenceTransformer : 모델 훈련 위한 라이브러리
# 'distiluse-base-multilingual-cased-v1' : 사용할 모델
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [4]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
print(len(keyword))

102


In [5]:
#키워드 벡터화
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

print(len(keyword_embedding))

102


In [6]:
#호주 문서 10개 embedding
import nltk
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

doc_embedding = []
doc_words = []
cnt = 1

for doc in df_add:
    result = []
    sentence = ''
    stop_words = set(stopwords.words('english'))
    
    doc = doc.replace("\n", "").strip()

    word_tokens = pos_tag(word_tokenize(doc))

    word_tokens = [t[0] for t in word_tokens if t[1] == "NN" and len(t[0]) > 2]
    for w in word_tokens:  
        if w not in stop_words:
            result.append(w)
    result = [word.lower() for word in result]
    doc_words.append(result)
    print(result)
    doc_embedding.append(model.encode(result))
    break
    #카운트 추가
    print(cnt, "번 문서 진행 중")
    cnt += 1 

['indexation', 'duty', 'indexation', 'duty', 'notice', 'duty', 'fuel', 'section', 'duty', 'application', 'indexation', 'factor', 'indexation', 'factor', 'number', 'number', 'figure', 'indexation', 'factor', 'duty', 'rate', 'increase', 'figure', 'refer', 'figure', 'indexation', 'factor', 'table', 'number', 'indexation', 'factor', 'duty', 'application', 'factor', 'duty', 'indexation', 'schedule', 'service']


In [17]:
#키워드와 문서 유사도 비교 0.9
doc_keywords_9 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.9:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_9.append(set(final_result))

set()
{'car'}
set()
set()
set()
set()
set()
set()
{'plastic'}
set()


In [18]:
#키워드와 문서 유사도 비교 0.8
doc_keywords_8 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.8:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_8.append(set(final_result))

set()
{'car', 'measure'}
set()
set()
set()
set()
set()
set()
{'plastic', 'measure'}
set()


In [19]:
#키워드와 문서 유사도 비교 0.75
doc_keywords_75 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.75:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_75.append(set(final_result))

{'fuel'}
{'car', 'measure', 'fuel'}
set()
set()
set()
set()
set()
set()
{'plastic', 'measure', 'fuel'}
set()


In [20]:
df_check = pd.DataFrame()
df_check['cosine 0.9'] = doc_keywords_9
df_check['cosine 0.8'] = doc_keywords_8
df_check['cosine 0.75'] = doc_keywords_75

df_check

,cosine 0.9,cosine 0.8,cosine 0.75
0,{},{},{fuel}
1,{car},"{car, measure}","{car, measure, fuel}"
2,{},{},{}
3,{},{},{}
4,{},{},{}
5,{},{},{}
6,{},{},{}
7,{},{},{}
8,{plastic},"{plastic, measure}","{plastic, measure, fuel}"
9,{},{},{}


In [10]:
# df['cosine 0.9'] = doc_keywords_9
# df['cosine 0.8'] = doc_keywords_8
# df['cosine 0.75'] = doc_keywords_75

# df

In [11]:
# df.to_csv("호주_세관_최근10_유사도, 번역 추가.csv")